# Ejercicio Cassansdra

## Gestión de reservas de hoteles

Un hotel desea gestionar huéspedes en un libro de reservas. 

Nuestro dominio conceptual contiene hoteles, huéspedes que se alojan en los hoteles, un conjunto de habitaciones para cada hotel, y un registro de las reservas. 

Los hoteles también mantienen una colección de puntos de interés los cuales son museos, parques, almacenes de compras, monumentos u otros lugares cercanos al hotel que los huéspedes quieran visitar durante su estancia. 

Tanto hoteles como puntos de interés necesitan mantener datos de localización geográfica para ser localizados en mapas y para calcular distancias.


### 1. Conectamos al cluster de Cassandra y preparamos el keyspace para el ejercicio

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=['127.0.0.1'], port=9042, auth_provider=auth_provider)
session = cluster.connect()

In [ ]:
session.execute("DROP KEYSPACE IF EXISTS hotels")

session.execute("CREATE KEYSPACE hotels WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1}")
session.set_keyspace('hotels')

### 2. Creamos las tablas para los datos

In [ ]:
#1.. Encontrar hoteles de una ciudad concreta de un país concreto.

session.execute(
    """
    CREATE TABLE hotels_by_city_and_country (
    country text, 
    city text, 
    name text,  
    category text, 
    PRIMARY KEY (country, city))
    """
)

In [ ]:
#2.. Encontrar información acerca de un hotel específico como nombre y localización.

session.execute(
    """
    CREATE TABLE hotels (
    name text, 
    country text, 
    city text,
    PRIMARY KEY (name))
    """
)

In [ ]:
#3.. Encontrar puntos de interés más cercanos al hotel específico ordenados por distancia.

session.execute(
    """
    CREATE TABLE points_of_interest_by_distance (
    hotel_name text,
    name text,
    distance float, 
    point_type text, 
    address text, 
    PRIMARY KEY (hotel_name, distance, point_type))
    """
)

In [ ]:
#4.. Encontrar puntos de interés más cercanos al hotel específico ordenados por distancia y en función de su categoría. 
#    Por ejemplo, los monumentos más cercanos.

session.execute(
    """
    CREATE TABLE points_of_interest_by_type_and_distance (
    hotel_name text,
    name text, 
    point_type text,
    distance float,  
    address text, 
    PRIMARY KEY (hotel_name, point_type, distance))
    """
)

In [ ]:
#5.. Encontrar una habitación disponible en una fecha dada.

session.execute(
    """
    CREATE TABLE rooms_by_date (
    date date, 
    free boolean,
    hotel_name text,  
    room int, 
    PRIMARY KEY (date, hotel_name, room, free))
    """
)

In [ ]:
#6.. Encontrar precios y características de las habitaciones.

session.execute(
    """
    CREATE TABLE rooms (
    room int, 
    price float,
    info text,  
    PRIMARY KEY (room, price))
    """
)

In [ ]:
#7.. Reservar la habitación seleccionada agregando los datos del huésped.

session.execute(
    """
    CREATE TABLE reservations (
    hotel text, 
    room int,
    full_name text,  
    address text, 
    PRIMARY KEY (hotel, room))
    """
)

### 3. Procesamos los datos y los insertamos en las tablas correspondientes

In [ ]:
import pandas as pd

df_hotels = pd.read_csv("../data/hotels.csv", header = 0)
df_rooms = pd.read_csv("../data/rooms.csv", header = 0)
df_reservations = pd.read_csv("../data/reservations.csv", header = 0)
df_points = pd.read_csv("../data/points.csv", header = 0)

In [ ]:
print(df_hotels.info())
print(df_hotels.head(10))

In [ ]:
for index, hotel in df_hotels.head(100).iterrows():
    session.execute(
        """
        INSERT INTO hotels_by_city_and_country (country, city, name, category)
        VALUES (%s, %s, %s, %s)
        """,
        (hotel['country'], hotel['city'], hotel['name'], str(hotel['category']))
    )

In [ ]:
for index, hotel in df_hotels.head(100).iterrows():
    session.execute(
        """
        INSERT INTO hotels (name, country, city)
        VALUES (%s, %s, %s)
        """,
        (hotel['name'], hotel['country'], hotel['city'])
    )

In [ ]:
poits_hotel = pd.merge(df_hotels, df_points, on=['id'])
print(poits_hotel.info())
print(poits_hotel.head(10))

In [ ]:
for index, point in poits_hotel.head(100).iterrows():
    session.execute(
        """
        INSERT INTO points_of_interest_by_distance (hotel_name, name, distance, point_type, address)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (point['name_x'], point['name_y'], point['distance'], point['category_y'], point['address'])
    )

In [ ]:
for index, point in poits_hotel.head(100).iterrows():
    session.execute(
        """
        INSERT INTO points_of_interest_by_type_and_distance (hotel_name, name, distance, point_type, address)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (point['name_x'], point['name_y'], point['distance'], point['category_y'], point['address'])
    )

In [ ]:
print(df_rooms.info())
print(df_rooms.head(10))

In [ ]:
print(df_reservations.info())
print(df_reservations.head(10))

In [ ]:
hotels_with_reservations = pd.merge(df_hotels, df_reservations, on=['id'])
print(hotels_with_reservations.info())
print(hotels_with_reservations.head(10))

In [ ]:
for index, room in hotels_with_reservations.head(100).iterrows():
    session.execute(
        """
        INSERT INTO rooms_by_date (date, free, hotel_name, room)
        VALUES (%s, %s, %s, %s)
        """,
        (room['date'], room['free'], room['name'], room['roomNumber'])
    )

In [ ]:
hotels_with_room = pd.merge(df_hotels, df_rooms, on=['id'])
print(hotels_with_room.info())
print(hotels_with_room.head(10))

In [ ]:
for index, room in hotels_with_room.head(100).iterrows():
    session.execute(
        """
        INSERT INTO rooms (room, price, info)
        VALUES (%s, %s, %s)
        """,
        (room['number'], room['price'], room['info'])
    )

In [ ]:
for index, reservation in hotels_with_reservations.head(100).iterrows():
    session.execute(
        """
        INSERT INTO reservations (hotel, room)
        VALUES (%s, %s)
        """,
        (reservation['name'], reservation['roomNumber'])
    )

In [ ]:
#Vemos como han quedado las tablas

hotels_by_city_and_country = session.execute("SELECT * FROM hotels_by_city_and_country LIMIT 10")
for row in hotels_by_city_and_country:
    print(row)   

In [ ]:
hotels = session.execute("SELECT * FROM hotels LIMIT 10")
for row in hotels:
    print(row)

In [ ]:
points_of_interest_by_distance = session.execute("SELECT * FROM points_of_interest_by_distance LIMIT 10")
for row in points_of_interest_by_distance:
    print(row)

In [ ]:
points_of_interest_by_type_and_distance = session.execute("SELECT * FROM points_of_interest_by_type_and_distance LIMIT 10")
for row in points_of_interest_by_type_and_distance:
    print(row) 

In [ ]:
rooms_by_date = session.execute("SELECT * FROM rooms_by_date LIMIT 10")
for row in rooms_by_date:
    print(row) 

In [ ]:
rooms = session.execute("SELECT * FROM rooms LIMIT 10")
for row in rooms:
    print(row)

In [ ]:
reservations = session.execute("SELECT * FROM reservations LIMIT 10")
for row in reservations:
    print(row)

### 4. Ejecutamos las queries

In [ ]:
#1.. Encontrar hoteles de una ciudad concreta de un país concreto.

result = session.execute("SELECT * FROM hotels_by_city_and_country WHERE country = 'Italy' AND city = 'Florencia'")

for row in result:
    print(row)

In [ ]:
#2.. Encontrar información acerca de un hotel específico como nombre y localización.

result_hotels = session.execute("SELECT * FROM hotels WHERE name = 'Maison Lavapies 100'")

for row in result_hotels:
    print(row)

In [ ]:
#3.. Encontrar puntos de interés más cercanos al hotel específico ordenados por distancia.

result_points_distance = session.execute("SELECT * FROM points_of_interest_by_distance WHERE hotel_name = 'Parador Magenta 5'")

for row in result_points_distance:
    print(row)

In [ ]:
#4.. Encontrar puntos de interés más cercanos al hotel específico ordenados por distancia y en función de su categoría. 

result_points_type = session.execute("SELECT * FROM points_of_interest_by_type_and_distance WHERE hotel_name = 'Parador Magenta 5'")

for row in result_points_type:
    print(row)

In [ ]:
#5.. Encontrar una habitación disponible en una fecha dada.

result_date = session.execute("SELECT * FROM rooms_by_date WHERE date = '2018-01-12'")

for row in result_date:
    print(row)

In [ ]:
#6.. Encontrar precios y características de las habitaciones.

result_room = session.execute("SELECT * FROM rooms WHERE room = 12")

for row in result_room:
    print(row)

In [ ]:
#7.. Reservar la habitación seleccionada agregando los datos del huésped.

result_room = session.execute("SELECT * FROM reservations WHERE hotel = '' AND room = ' '")

for row in result_room:
    print(row)